In [1]:
import sys
sys.path.append('network')
import network
import svg_draw as sd
import pandas as pd

def draw_network(adj_df, eigen_df, layout_df, max_edge_width = 2, max_r = 10, min_width = 1, cutoff=0.9):
    origin = (430, 430)
    common_taxon = list(set(adj_df.index).intersection(set(layout_df.index)))
    adj_df = adj_df.loc[common_taxon, common_taxon]
    circles = ''
    eigen = ''
    pr_scores = eigen_df['pr']
    pr_max = max(pr_scores)
    pr_min = min(pr_scores)
    min_r = 5
    pr_range = pr_max-pr_min
    xy_df = pd.DataFrame()
    for idx in common_taxon:
        theta = layout_df.loc[idx, 'theta']
        r = layout_df.loc[idx, 'r']
        color = layout_df.loc[idx, 'color']
        x, y = network.convert(theta, r, origin)
        xy_df.loc[idx, 'x'] = x
        xy_df.loc[idx, 'y'] = y
        pr = eigen_df.loc[idx, 'pr']
        if eigen_df.loc[idx, 'eigen'] == True:
            eigen = idx
            circle = sd.draw_circle(x, y, r=max_r, fill="black", id=idx, swidth=0, style=True)
        else:
            r = (max_r-min_r)*(pr-pr_min)/pr_range  + min_r
            circle = sd.draw_circle(x, y, r=r, fill=color, id=idx, swidth=0, style=True)
        circles +=  circle
    
    paths = ""
    ntaxon = adj_df.shape[0]
    taxon = list(adj_df.index)
    for i in range(ntaxon):
        taxa1 = taxon[i]
        x1 = xy_df.loc[taxa1, 'x']
        y1 = xy_df.loc[taxa1, 'y']
        for j in range(i+1, ntaxon):
            taxa2 = taxon[j]
            cor_value = adj_df.loc[taxa1, taxa2]
            if cor_value > cutoff:
                x2 = xy_df.loc[taxa2, 'x']
                y2 = xy_df.loc[taxa2, 'y']
                id = "{}_{}".format(taxa1, taxa2)
                x3, y3 = network.compute_convex((x1, y1), (x2, y2))
                edge_width = max_edge_width*cor_value
                edge_width = 1
                if taxa1 == eigen or taxa2 == eigen:
                    path = sd.draw_curve(x1, y1, x3, y3, x2, y2, stroke='red', swidth=edge_width, id=id, style=True)
                else:
                    path = sd.draw_curve(x1, y1, x3, y3, x2, y2, stroke= '#AAAAAA', swidth=edge_width, id=id, style=True)
                paths += path
    title = '<text x="0" y="20">{}</text>'.format(eigen)
    title = ''
    return title + paths + circles




In [2]:
legend = '<circle cx="150" cy="860" r="12" stroke="black" stroke-width="1" fill="black" /> \n \
<text x="180" y="865" class="legend">Largest PR</text>\n \
<circle cx="350" cy="860" r="12" stroke="black" stroke-width="1" fill="#1B9E77" />\n \
<text x="380" y="865" class="legend">p__Firmicutes_A</text>\n \
<circle cx="550" cy="860" r="12" stroke="black" stroke-width="1" fill="#D95F02" />\n \
<text x="580" y="865" class="legend">p__Bacteroidota</text>\n \
<circle cx="750" cy="860" r="12" stroke="black" stroke-width="1" fill="#7570B3" />\n \
<text x="780" y="865" class="legend">p__Firmicutes</text>\n \
<circle cx="150" cy="890" r="12" stroke="black" stroke-width="1" fill="#E7298A" />\n \
<text x="180" y="895" class="legend">p__Proteobacteria</text>\n \
<circle cx="350" cy="890" r="12" stroke="black" stroke-width="1" fill="#66A61E" />\n \
<text x="380" y="895" class="legend">p__Actinobacteriota</text>\n \
<circle cx="550" cy="890" r="12" stroke="black" stroke-width="1" fill="#E6AB02" />\n \
<text x="580" y="895" class="legend">p__Firmicutes_C</text>\n \
<circle cx="750" cy="890" r="12" stroke="black" stroke-width="1" fill="#A6761D" />\n \
<text x="780" y="895" class="legend">p__Verrucomicrobiota</text>\n \
<circle cx="150" cy="920" r="12" stroke="black" stroke-width="1" fill="#E41A1C" />\n \
<text x="180" y="925" class="legend">p__Firmicutes_B</text>\n \
<circle cx="350" cy="920" r="12" stroke="black" stroke-width="1" fill="#377EB8" />\n \
<text x="380" y="925" class="legend">p__Cyanobacteria</text>\n \
<circle cx="550" cy="920" r="12" stroke="black" stroke-width="1" fill="#FFFF33" />\n \
<text x="580" y="925" class="legend">p__Fusobacteriota</text>\n \
<circle cx="750" cy="920" r="12" stroke="black" stroke-width="1" fill="#AAAAAA" />\n \
<text x="780" y="925" class="legend">Other</text>'

legend = ''

def draw(content, legend):
    width = 860
    height = 860
    return sd.canvas(width, height, content+legend)



In [3]:
import os
from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg
layout_df = pd.read_csv('sector_sp_layout.tsv', sep='\t', header=0, index_col=0)
outer_dir = 'result'

for d in os.listdir(outer_dir):
    in_dir = os.path.join(outer_dir, d)
    if not os.path.isdir(in_dir):
        continue
   
    
    eigen_df = pd.read_csv(os.path.join(in_dir, 'eigen.tsv'), sep='\t', header=0, index_col=0)
    fr_df = pd.read_csv(os.path.join(in_dir, 'adj.tsv'), sep='\t', header=0, index_col=0)
    s_sector= draw(draw_network(fr_df, eigen_df, layout_df, cutoff=0.1), legend)
    svg_dir = os.path.join(in_dir, 'network.svg')
    with open(svg_dir, 'w') as fp:
        fp.write(s_sector)

    drawing = svg2rlg(svg_dir)
    pdf_path = os.path.join(in_dir, 'network.pdf')
    renderPDF.drawToFile(drawing, pdf_path)

            